# Basics of Mobile Robotics - Main Notebook

Catarina Pires #SIPHER, Dimitri Hollosi #SIPHER, Gonçalo Gomes #SIPHER, Richard Gao #SIPHER

In [ ]:
# Import libraries
import numpy as np
import Tracking as TR
from tdmclient import ClientAsync
# Import tdmclient Notebook environment:
# import tdmclient.notebook
# await tdmclient.notebook.start()
# tdmclient.notebook.stop()

In [ ]:
client = ClientAsync()
node = client.aw(client.wait_for_node())

# Thymio Parametres

In [ ]:
r = 2 # radius of the wheel (cm)
L = 9.5 # distance from wheel to wheel (cm)
v_max = 20 # cm/s
cruising = 150 
motor_speed_max = 500
omega_to_motor = (motor_speed_max*r)/v_max

# Controller Parametres

In [ ]:
Kp = 10   # Proporcional gain
Ki = 0.1  # Integral gain
Kd = 1    # Derivative gain
error_sum = 0
previous_error = 0

# Kalman Filter Parametres

In [ ]:
ts = 0.1  # time step
A = np.array([[1, ts, 0, 0, 0, 0],  # state transition matrix
              [0, 1, 0, 0, 0, 0],
              [0, 0, 1, ts, 0, 0],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, ts],
              [0, 0, 0, 0, 0, 1]])

P0 = TR.Tracking.covariance(20, 5, 20, 5, 20, 5)  # state covariance matrix

Q = TR.Tracking.covariance(20, 5, 20, 5, 20, 5)  # process noise covariance matrix

R = TR.Tracking.covariance(25, 6, 25, 6, 25, 6)  # measurement covariance matrix

In [ ]:
#---------------------------------------------------------------------------
#                                 Computer vision Part
#---------------------------------------------------------------------------


# Inicial states
X0 = np.array([[source_x],  # x0
                    [0],    # x_dot0
              [source_y],   # y0
                    [0],    # y_dot0
                [phi_0],    # phi
                    [0]])   # phi_dot

In [ ]:
# Define local_nav object with class attributes
tracking = TR.Tracking(client, node, client, node, cruising, ts, Kp, Ki, Kd, error_sum, previous_error, r, L, omega_to_motor, path, R, Q, A, X0, P0)


In [ ]:
#----------------------------------------------------
# Computer Vision part and getting the shortest path
#---------------------------------------------------
# while (...)
#        u: desired velocity u=[u1, u2] from the planner
#        phi: current angle of the thymio

phi_d = np.artcan((y_goal-y_source)/(x_goal-x_source))
if phi_d <= 0.1:  
    v = 0 # we define a constant velocity for the thymio until it reaches the goal and just control teh orientation
else:
    v = v_cruising
#                                                      phi from kalman
vr, vl, new_error_sum, new_previous_error = PIDcontroller(phi_d, phi, Kp, Ki, Kd, error_sum, previous_error, R, L, v)
error_sum = new_error_sum
previous_error = new_previous_error

motor_left = vl*omega_to_motor
motor_right = vr*omega_to_motor

# Saturation
if motor_left > 500:
    motor_left = 500

elif motor_right > 500:
    motor_right = 500

elif motor_right < -500:
    motor_right = -500

elif motor_left < -500:
    motor_left = -500

motor_left_target = motor_left
motor_right_target = motor_right


# Update states through sensor data ////////////////////////
# -> get motor_left_speed and motor_right_speed from thymio
vr_thymio = motor_right_speed/omega_to_motor
vl_thymio = motor_left_speed/omega_to_motor

phidot_obs[i+1] = (R/L)*(vr_thymio-vl_thymio)
xdot_obs = (R/2)*(vr_thymio + vl_thymio)*np.cos(phi_obs[i+1])
ydot_obs = (R/2)*(vr_thymio + vl_thymio)*np.sin(phi_obs[i+1])

# if camera available:
#    -> get x, y
# else:
phi_obs[i+1] = np.arctan(ydot_obs/xdot_obs)
x_obs[i+1] = x_obs[i] + xdot_obs*time_step
y_obs[i+1] = y_obs[i] + ydot_obs*time_step

X_est, P_est = kl.kalman_filter(X_est, P_est, Q, R, x_obs[i+1], xdot_obs[i+1], y_obs[i+1], ydot_obs[i+1], phi_obs[i+1], phidot_obs[i+1])
